In [163]:
import pandas as pd
import matplotlib.pyplot as plt
import akshare as ak
import datetime
import glob
import numpy as np

# 获取数据

In [164]:
def get_data(index="sh510050"):
    sh50_files = sorted(glob.glob("../sh_etf_data/{}_*.5.tsv".format(index)))

    sh50_pd = pd.read_table(sh50_files[0], header=0, index_col=0,parse_dates=["day"])
    for f in sh50_files[1:]:
        add_pd = pd.read_table(f, header=0, index_col=0,parse_dates=["day"])
        add_pd = add_pd[add_pd["day"] > list(sh50_pd["day"])[-1]] 
        sh50_pd = sh50_pd.append(add_pd)



    sh50_pd.reset_index(drop=True, inplace=True)
    sh50_pd["close_yes"] = sh50_pd["close"].shift()
    sh50_pd = sh50_pd[sh50_pd["day"].map(str) > "2022-10-28"]
    return sh50_pd


# 获取n日后的价格

In [165]:
def finish_data(data_df):
    data_df["close_2hour"] = data_df["close"].shift(-24)
    data_df["close_1day"] = data_df["close"].shift(-48)
    data_df["close_2day"] = data_df["close"].shift(-48*2)
    data_df["close_3day"] = data_df["close"].shift(-48*3)
    data_df["close_5day"] = data_df["close"].shift(-48*5)
    data_df["close_10day"] = data_df["close"].shift(-48*10)

    
    return data_df

# 获取N日后收益率

In [211]:
def day_data(data_df):
    first5min_pd = data_df.iloc[0::48, :]
    first5min_pd = first5min_pd.reset_index()
    first5min_pd["volume_change"] = first5min_pd["volume"].pct_change()
    first5min_pd["close_change_yes"] = (first5min_pd["open"] - first5min_pd["close_yes"])/first5min_pd["close_yes"]
    first5min_pd["close_change_2h"] = (first5min_pd["close_2hour"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change1"] = (first5min_pd["close_1day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change2"] = (first5min_pd["close_2day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change3"] = (first5min_pd["close_3day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change5"] = (first5min_pd["close_5day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change10"] = (first5min_pd["close_10day"] - first5min_pd["close"])/first5min_pd["close"]
    
    return first5min_pd

In [212]:
def mean_value(first5min_pd):
    #每次交易收益率
    a02 = np.mean(first5min_pd["close_change_2h"])
    a1 = np.mean(first5min_pd["close_change1"])
    a2 = np.mean(first5min_pd["close_change2"])
    a3 = np.mean(first5min_pd["close_change3"])
    a5 = np.mean(first5min_pd["close_change5"])
    a10 = np.mean(first5min_pd["close_change10"])
    return [a02, a1, a2, a3, a5, a10]

In [283]:
def get_extre(index="sz159915"):
    sh50_pd = get_data(index=index)
    sh50_pd = finish_data(sh50_pd)
    day_data1 = day_data(sh50_pd)
    a = len(day_data1)
    origin = mean_value(day_data1)
    # 策略获取首5分钟内放量超过10%的交易
    first5min_choosefangliang_pd = day_data1[day_data1["volume_change"] > 0.05 ]
#     first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close"] > first5min_choosefangliang_pd["open"] ]
    #高开  1%
    first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close_change_yes"] < -0.005 ]
 
    choose = mean_value(first5min_choosefangliang_pd)
    b = len(first5min_choosefangliang_pd)
    d = ";".join(list(first5min_choosefangliang_pd["day"].map(str)))
    return([x-y for x,y in zip(choose, origin)] + [a, b, d])



In [214]:
len(day_data1[day_data1["close_change_2h"]> 0.01])

12

In [377]:
def get_pct_oneday(index="sz159915"):
    sh50_pd = get_data(index=index)
    sh50_pd = finish_data(sh50_pd)
    day_data1 = day_data(sh50_pd)
    get = "get_" + index
    day_data1[get] = 0
    choose = (day_data1["volume_change"] > 0.05) & (day_data1["close_change_yes"] < -0.005)
    choose_nextday = choose.shift().fillna(False)
    day_data1[get][choose | choose_nextday] = day_data1["close_change1"]

    return day_data1

In [379]:
tmp = get_pct_oneday(index="sh510050")
tmp

/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]


,index,day,open,high,low,close,volume,close_yes,close_2hour,close_1day,...,close_10day,volume_change,close_change_yes,close_change_2h,close_change1,close_change2,close_change3,close_change5,close_change10,get_sh510050
0,15,2022-10-28 09:35:00,2.403,2.413,2.393,2.393,67370900,2.419,2.396,2.332,...,2.547,NaN,-0.006614,0.001254,-0.025491,-0.015462,0.012537,0.025073,0.064354,0.000000
1,63,2022-10-31 09:35:00,2.344,2.348,2.326,2.332,112566800,2.364,2.346,2.356,...,2.606,0.670852,-0.008460,0.006003,0.010292,0.039022,0.046312,0.072899,0.117496,0.010292
2,111,2022-11-01 09:35:00,2.342,2.362,2.342,2.356,116989143,2.333,2.385,2.423,...,2.565,0.039286,0.003858,0.012309,0.028438,0.035654,0.041171,0.064941,0.088710,0.028438
3,159,2022-11-02 09:35:00,2.422,2.425,2.415,2.423,70515372,2.437,2.459,2.440,...,2.617,-0.397249,-0.006155,0.014858,0.007016,0.012381,0.032604,0.032191,0.080066,0.000000
4,207,2022-11-03 09:35:00,2.438,2.445,2.432,2.440,66175229,2.456,2.415,2.453,...,2.603,-0.061549,-0.007329,-0.010246,0.005328,0.025410,0.028279,0.008197,0.066803,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,4095,2023-03-06 09:35:00,2.779,2.780,2.767,2.768,34372889,2.779,2.748,2.751,...,NaN,-0.167777,0.000000,-0.007225,-0.006142,-0.022038,-0.027818,NaN,NaN,0.000000
86,4143,2023-03-07 09:35:00,2.744,2.752,2.736,2.751,25956700,2.749,2.752,2.707,...,NaN,-0.244850,-0.001819,0.000364,-0.015994,-0.021810,-0.029807,NaN,NaN,0.000000
87,4191,2023-03-08 09:35:00,2.708,2.710,2.701,2.707,55862300,2.719,2.699,2.691,...,NaN,1.152134,-0.004046,-0.002955,-0.005911,-0.014038,NaN,NaN,NaN,0.000000
88,4239,2023-03-09 09:35:00,2.698,2.700,2.690,2.691,22722500,2.697,2.680,2.669,...,NaN,-0.593241,0.000371,-0.004088,-0.008175,NaN,NaN,NaN,NaN,0.000000


In [372]:
a["index"][(a["volume_change"] > 0.05) & (a["close_change_yes"] < -0.005) ] = 10

/tmp/ipykernel_2864734/1278070732.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a["index"][(a["volume_change"] > 0.05) & (a["close_change_yes"] < -0.005) ] = 10


In [314]:
merge_df = tmp[["day"]]
merge_df
index = "sh512100"
a = get_pct_oneday(index=index)
index_df = a[["day", "get_" + index]]
index_df
merge_df = pd.merge(left=merge_df, right=index_df, left_on=["day"], right_on=["day"])
merge_df

/tmp/ipykernel_2864734/2280532093.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][(day_data1["volume_change"] > 0.05) & (day_data1["close_change_yes"] < -0.005) ] = day_data1["close_change1"]


,day,get_sh512100
0,2022-10-28 09:35:00,0.000000
1,2022-10-31 09:35:00,0.016854
2,2022-11-01 09:35:00,0.000000
3,2022-11-02 09:35:00,0.000000
4,2022-11-03 09:35:00,0.008819
...,...,...
85,2023-03-06 09:35:00,0.000000
86,2023-03-07 09:35:00,0.000000
87,2023-03-08 09:35:00,0.002188
88,2023-03-09 09:35:00,0.000000


In [380]:
index_list = [
    "sh510050", "sh512100", "sh563000", "sh510300", "sh510330", 
    "sh510500", "sh561550",
    "sz159845", "sz159949",  "sz159915", "sz159967", "sh588000", "sh588080",
    "sh588050", "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", "sh516160", 
    "sh515030", "sh515700", "sh515050", "sh512880", "sh512000", 
    "sh512070", "sh512800", "sh512200", "sh510880", 
    "sz159825", "sz159867",  "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", 
    "sh515030", "sh515700", "sz159611", "sh515050", "sh512880",
    "sh512000", "sh512070", "sh512800", "sh512200", "sh510880", "sz159905"
]
result = []
merge_df = tmp[["day"]]
for index in index_list:
    print(index)
    print(len(merge_df))
    a = get_pct_oneday(index=index)
    if len(a) < 90:
        continue
    index_df = a[["day", "get_" + index]]
    merge_df = pd.merge(left=merge_df, right=index_df, left_on=["day"], right_on=["day"])

sh510050
90
sh512100
90
sh563000
90
sh510300
90


/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = d

sh510330
90
sh510500
90
sh561550
90
sz159845
90


/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = d

sz159949
90
sz159915
90
sz159967
90
sh588000
90


...

/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = d

sz159611
90
sh515050
90
sh512880
90
sh512000
90


/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = d

sh512070
90
sh512800
90
sh512200
90
sh510880
90
sz159905
90


/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]
/tmp/ipykernel_2864734/2451935735.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  day_data1[get][choose | choose_nextday] = day_data1["close_change1"]


In [359]:
a[]

SyntaxError: invalid syntax (1145722274.py, line 1)

In [376]:
a1 = (a["open"] > 1.6).shift().fillna(False)
a2 = a1.shift().fillna(False)
a1 | a2

0     False
1      True
2      True
3      True
4      True
      ...  
85     True
86     True
87     True
88     True
89     True
Name: open, Length: 90, dtype: bool

In [388]:
mean_get = list()
len_get = list()
for day_raw in merge_df.iterrows():
    day_dict = day_raw[1]
    etfs = list()
    gets = list()
    for k,v in day_dict.items():
        if k.startswith("get") and  v != 0.0:
            etfs.append(k)
            gets.append(v)
    
    # print(etfs, gets)
    if len(gets) <=4:
        mean_get.append(0)
    else:
        mean_get.append(np.mean(gets))
    len_get.append(len(gets))
mean_get


[0,
 0.013407754514117748,
 0.0229056453863527,
 0,
 0.009923937898892848,
 0.02918639337277497,
 -0.002134605691658561,
 0.003954068053913422,
 0,
 0.012816093615765665,
 -0.0071725349452756906,
 0,
 0,
 0,
 -0.0029808040490434972,
 -0.01261191745895288,
 0.0020838431957937264,
 -0.0005820468781612115,
 0,
 0,
 0,
 0.01744519617153984,
 0.014708552916476509,
 0,
 0,
 0,
 0,
 0,
 -0.0034591308684070797,
 0.013596217873075975,
 0,
 -0.006146668862156699,
 0.010062403503318563,
 0,
 0,
 0.0011368696347489552,
 -0.020629449563477845,
 0,
 0,
 0,
 0.002104780006988249,
 0.025955640829089178,
 0,
 -0.0018181779941204767,
 0.005919166260759112,
 -0.00360890848839892,
 0.016417924650599713,
 0.008706921012881416,
 0.01782713239106881,
 0,
 0,
 0,
 0,
 0,
 0,
 0.023411557771199992,
 0.004896835937812906,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.0005136811181571256,
 0.002010297141582304,
 0,
 0,
 -0.006749998210339213,
 0.006296359032207359,
 0,
 0,
 0,
 0,
 0,
 0,
 -0.002221047942474511,
 0.0008455

In [389]:
merge_df["mean_get"] = mean_get
merge_df["len_get"] = len_get
merge_df["mean_get_cum"] = (merge_df["mean_get"] + 1).cumprod()
merge_df.to_csv("all_etf_get.tsv", sep="\t")

In [390]:
(merge_df["mean_get"] + 1).cumprod()

0     1.000000
1     1.013408
2     1.036621
3     1.036621
4     1.046908
        ...   
85    1.221858
86    1.221858
87    1.219802
88    1.211669
89         NaN
Name: mean_get, Length: 90, dtype: float64

In [271]:
index_list = [
    "sh510050", "sh512100", "sh563000", "sh560050", "sh510300", "sh510330", 
    "sh561300", "sh561990", "sh510500", "sh561550", "sh512500",
    "sz159845", "sz159949",  "sz159915", "sz159967", "sh588000", "sh588080",
    "sh588050", "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", "sh516160", 
    "sh515030", "sh515700", "sz159611", "sh516010", "sh515050", "sh512880", "sh512000", 
    "sh512070", "sh512800", "sh512200", "sh510880", "sz159905", "sh516950", "sz159745", 
    "sz159825", "sz159867",  "sh515790", "sh512480", "sh512760", "sz159995", "sh512660", 
    "sh516160", "sh515030", "sh515700", "sz159611", "sh516010", "sh515050", "sh512880",
    "sh512000", "sh512070", "sh512800", "sh512200", "sh510880", "sz159905", "sh516950",
    "sz159745", "sz159825", "sz159867"
]
result = []
for index in index_list:
    re = get_extre(index=index)
    result.append(re)

In [272]:
a = pd.DataFrame(result)
print(a.mean())
a

0     0.000688
1     0.005104
2     0.008931
3     0.009266
4     0.008499
5     0.011797
6    89.967742
7     5.919355
dtype: float64


/tmp/ipykernel_2864734/2045308912.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  print(a.mean())


,0,1,2,3,4,5,6,7,8
0,-0.000107,0.009979,0.017273,0.019109,0.019418,0.029134,90,10,2022-10-31 09:35:00;2022-11-10 09:35:00;2022-1...
1,0.002474,0.006374,0.015125,0.020479,0.032749,0.020949,90,7,2022-10-31 09:35:00;2022-11-03 09:35:00;2022-1...
2,-0.000118,0.007032,0.009936,0.004110,-0.000781,0.007179,90,7,2022-11-10 09:35:00;2022-11-21 09:35:00;2022-1...
3,0.003650,0.006639,-0.006163,-0.004593,0.002674,0.023561,90,3,2022-11-02 09:35:00;2022-11-23 09:35:00;2022-1...
4,0.001052,0.008858,0.018299,0.015589,0.021216,0.021076,90,9,2022-10-31 09:35:00;2022-11-03 09:35:00;2022-1...
...,...,...,...,...,...,...,...,...,...
57,0.006174,0.012111,0.011929,0.018355,0.005872,0.032569,90,5,2022-11-21 09:35:00;2022-11-28 09:35:00;2022-1...
58,NaN,NaN,NaN,NaN,NaN,NaN,90,0,
59,-0.005182,-0.004111,0.004881,0.004230,0.004282,0.041662,90,5,2022-10-31 09:35:00;2022-11-02 09:35:00;2022-1...
60,0.000613,0.005184,0.009379,0.011119,0.011807,-0.008114,90,3,2022-11-28 09:35:00;2022-12-12 09:35:00;2023-0...


In [273]:
index_dict = {
    "sh510050": "上证50ETF",
    "sh563000": "中国A50ETF",
    "sh560050": "MSCI中国A50ETF",
    "sh510300": "沪深300ETF",
    "sh510330": "300ETF基金",
    "sh561300": "300增强ETF",
    "sh561990": "沪深300增强ETF",
    "sh510500": "中证500ETF",
    "sh561550": "500增强ETF",
    "sh512500": "500ETF基金",
    "sz159845": "中证1000ETF",
    "sh512100": "中证1000ETF",
    "sz159949": "创业板50ETF",
    "sz159915": "创业板ETF",
    "sz159967": "创成长ETF",
    "sh588000": "科创50ETF",
    "sh588080": "科创板50ETF",
    "sh588050": "科创ETF",
    "sh515790": "光伏ETF",
    "sh512480": "半导体ETF",
    "sh512760": "芯片ETF",
    "sz159995": "芯片ETF",
    "sh512660": "军工ETF",
    "sh516160": "新能源ETF",
    "sh515030": "新能源车ETF",
    "sh515700": "新能车ETF",
    "sz159611": "电力ETF",
    "sh516010": "游戏ETF",
    "sh515050": "5GETF",
    "sh512880": "证券ETF",
    "sh512000": "券商ETF",
    "sh512070": "证券保险ETF",
    "sh512800": "银行ETF",
    "sh512200": "房地产ETF",
    "sh510880": "红利ETF",
    "sz159905": "深红利ETF",
    "sh516970": "基建50ETF",
    "sh516950": "基建ETF",
    "sz159745": "建材ETF",
    "sz159825": "农业ETF",
    "sz159865": "养殖ETF",
    "sz159867": "畜牧ETF",
    "sz159605": "中概互联ETF",
    "sh513180": "恒生科技指数ETF",
    "sh513130": "恒生科技ETF",
    "sh513010": "恒生科技30ETF",
    "sz159920": "恒生ETF",
    "sh510900": "H股ETF",
    "sh513060": "恒生医疗ETF",
    "sh512010": "医药ETF",
    "sh512170": "医疗ETF",
    "sh512290": "生物医药ETF",
    "sh512690": "酒ETF",
    "sz159928": "消费ETF",
    "sz159996": "家电ETF",
    "sz159766": "旅游ETF",
    "sh515220": "煤炭ETF",
    "sh515210": "钢铁ETF",
    "sh516780": "稀土ETF",
    "sh512400": "有色金属ETF",
    "sh511380": "可转债ETF",
    "sh513500": "标普500ETF",
    "sh513100": "纳指ETF",
}
index_name =  [index_dict[x] for x in index_list]

a.index= index_name

In [269]:
index_name

['上证50ETF',
 '中证1000ETF',
 '中国A50ETF',
 'MSCI中国A50ETF',
 '沪深300ETF',
 '300ETF基金',
 '300增强ETF',
 '沪深300增强ETF',
 '中证500ETF',
 '500增强ETF',
 '500ETF基金',
 '中证1000ETF',
 '创业板50ETF',
 '创业板ETF',
 '创成长ETF',
 '科创50ETF',
 '科创板50ETF',
 '科创ETF',
 '光伏ETF',
 '半导体ETF',
 '芯片ETF',
 '芯片ETF',
 '军工ETF',
 '新能源ETF',
 '新能源车ETF',
 '新能车ETF',
 '电力ETF',
 '游戏ETF',
 '5GETF',
 '证券ETF',
 '券商ETF',
 '证券保险ETF',
 '银行ETF',
 '房地产ETF',
 '红利ETF',
 '深红利ETF',
 '基建ETF',
 '建材ETF',
 '农业ETF',
 '畜牧ETF',
 '光伏ETF',
 '半导体ETF',
 '芯片ETF',
 '芯片ETF',
 '军工ETF',
 '新能源ETF',
 '新能源车ETF',
 '新能车ETF',
 '电力ETF',
 '游戏ETF',
 '5GETF',
 '证券ETF',
 '券商ETF',
 '证券保险ETF',
 '银行ETF',
 '房地产ETF',
 '红利ETF',
 '深红利ETF',
 '基建ETF',
 '建材ETF',
 '农业ETF',
 '畜牧ETF']

In [274]:
a.to_csv("all_etf_dikai_fangliang3.tsv", sep="\t")

In [275]:
day = list()
for dayl in list(a[8]):
    day.extend(dayl.split(";"))
day

['2022-10-31 09:35:00',
 '2022-11-10 09:35:00',
 '2022-11-21 09:35:00',
 '2022-11-28 09:35:00',
 '2022-12-12 09:35:00',
 '2022-12-23 09:35:00',
 '2022-12-29 09:35:00',
 '2023-02-06 09:35:00',
 '2023-02-27 09:35:00',
 '2023-03-10 09:35:00',
 '2022-10-31 09:35:00',
 '2022-11-03 09:35:00',
 '2022-11-28 09:35:00',
 '2022-12-29 09:35:00',
 '2023-02-06 09:35:00',
 '2023-03-08 09:35:00',
 '2023-03-10 09:35:00',
 '2022-11-10 09:35:00',
 '2022-11-21 09:35:00',
 '2022-12-23 09:35:00',
 '2022-12-26 09:35:00',
 '2023-02-06 09:35:00',
 '2023-02-22 09:35:00',
 '2023-02-27 09:35:00',
 '2022-11-02 09:35:00',
 '2022-11-23 09:35:00',
 '2022-12-16 09:35:00',
 '2022-10-31 09:35:00',
 '2022-11-03 09:35:00',
 '2022-11-07 09:35:00',
 '2022-11-10 09:35:00',
 '2022-11-21 09:35:00',
 '2022-11-28 09:35:00',
 '2023-01-16 09:35:00',
 '2023-02-06 09:35:00',
 '2023-03-10 09:35:00',
 '2022-11-21 09:35:00',
 '2022-11-28 09:35:00',
 '2023-01-16 09:35:00',
 '2023-02-06 09:35:00',
 '',
 '2022-11-07 09:35:00',
 '2022-11-2

In [276]:
set(day)

{'',
 '2022-10-31 09:35:00',
 '2022-11-02 09:35:00',
 '2022-11-03 09:35:00',
 '2022-11-07 09:35:00',
 '2022-11-10 09:35:00',
 '2022-11-15 09:35:00',
 '2022-11-16 09:35:00',
 '2022-11-17 09:35:00',
 '2022-11-18 09:35:00',
 '2022-11-21 09:35:00',
 '2022-11-23 09:35:00',
 '2022-11-25 09:35:00',
 '2022-11-28 09:35:00',
 '2022-12-05 09:35:00',
 '2022-12-06 09:35:00',
 '2022-12-07 09:35:00',
 '2022-12-12 09:35:00',
 '2022-12-13 09:35:00',
 '2022-12-15 09:35:00',
 '2022-12-16 09:35:00',
 '2022-12-19 09:35:00',
 '2022-12-20 09:35:00',
 '2022-12-23 09:35:00',
 '2022-12-26 09:35:00',
 '2022-12-28 09:35:00',
 '2022-12-29 09:35:00',
 '2023-01-03 09:35:00',
 '2023-01-04 09:35:00',
 '2023-01-05 09:35:00',
 '2023-01-16 09:35:00',
 '2023-02-03 09:35:00',
 '2023-02-06 09:35:00',
 '2023-02-10 09:35:00',
 '2023-02-13 09:35:00',
 '2023-02-17 09:35:00',
 '2023-02-20 09:35:00',
 '2023-02-22 09:35:00',
 '2023-02-27 09:35:00',
 '2023-03-08 09:35:00',
 '2023-03-10 09:35:00'}

每日取前5分钟数据

In [70]:
first5min_pd = sh50_pd.iloc[0::48, :]

In [25]:
first5min_pd

,day,open,high,low,close,volume,volume_yestday,close_1day,close_3day,close_10day
15,2022-10-28 09:35:00,2.403,2.413,2.393,2.393,67370900,NaN,2.332,2.423,2.547
63,2022-10-31 09:35:00,2.344,2.348,2.326,2.332,112566800,67370900.0,2.356,2.440,2.606
111,2022-11-01 09:35:00,2.342,2.362,2.342,2.356,116989143,112566800.0,2.423,2.453,2.565
159,2022-11-02 09:35:00,2.422,2.425,2.415,2.423,70515372,116989143.0,2.440,2.502,2.617
207,2022-11-03 09:35:00,2.438,2.445,2.432,2.440,66175229,70515372.0,2.453,2.509,2.603
...,...,...,...,...,...,...,...,...,...,...
3375,2023-02-13 09:35:00,2.746,2.753,2.738,2.752,34180900,32853261.0,2.774,2.760,NaN
3423,2023-02-14 09:35:00,2.776,2.784,2.768,2.774,21992500,34180900.0,2.781,2.759,NaN
3471,2023-02-15 09:35:00,2.785,2.791,2.780,2.781,22445477,21992500.0,2.760,NaN,NaN
3519,2023-02-16 09:35:00,2.764,2.768,2.759,2.760,23858300,22445477.0,2.759,NaN,NaN


In [71]:
first5min_pd["volume_change"] = first5min_pd["volume"].pct_change()
first5min_pd["close_change1"] = (first5min_pd["close_1day"] - first5min_pd["close"])/first5min_pd["close"]
first5min_pd["close_change2"] = (first5min_pd["close_2day"] - first5min_pd["close"])/first5min_pd["close"]
first5min_pd["close_change3"] = (first5min_pd["close_3day"] - first5min_pd["close"])/first5min_pd["close"]
first5min_pd["close_change5"] = (first5min_pd["close_5day"] - first5min_pd["close"])/first5min_pd["close"]
first5min_pd["close_change10"] = (first5min_pd["close_10day"] - first5min_pd["close"])/first5min_pd["close"]

/tmp/ipykernel_2864734/1281716087.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first5min_pd["volume_change"] = first5min_pd["volume"].pct_change()
/tmp/ipykernel_2864734/1281716087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first5min_pd["close_change1"] = (first5min_pd["close_1day"] - first5min_pd["close"])/first5min_pd["close"]
/tmp/ipykernel_2864734/1281716087.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [30]:
first5min_pd

,day,open,high,low,close,volume,volume_yestday,close_1day,close_3day,close_10day,volume_changge,volume_chang,volume_change,close_change1,close_change3,close_change10
15,2022-10-28 09:35:00,2.403,2.413,2.393,2.393,67370900,NaN,2.332,2.423,2.547,NaN,NaN,NaN,-0.025491,0.012537,0.064354
63,2022-10-31 09:35:00,2.344,2.348,2.326,2.332,112566800,67370900.0,2.356,2.440,2.606,0.670852,0.670852,0.670852,0.010292,0.046312,0.117496
111,2022-11-01 09:35:00,2.342,2.362,2.342,2.356,116989143,112566800.0,2.423,2.453,2.565,0.039286,0.039286,0.039286,0.028438,0.041171,0.088710
159,2022-11-02 09:35:00,2.422,2.425,2.415,2.423,70515372,116989143.0,2.440,2.502,2.617,-0.397249,-0.397249,-0.397249,0.007016,0.032604,0.080066
207,2022-11-03 09:35:00,2.438,2.445,2.432,2.440,66175229,70515372.0,2.453,2.509,2.603,-0.061549,-0.061549,-0.061549,0.005328,0.028279,0.066803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375,2023-02-13 09:35:00,2.746,2.753,2.738,2.752,34180900,32853261.0,2.774,2.760,NaN,0.040411,0.040411,0.040411,0.007994,0.002907,NaN
3423,2023-02-14 09:35:00,2.776,2.784,2.768,2.774,21992500,34180900.0,2.781,2.759,NaN,-0.356585,-0.356585,-0.356585,0.002523,-0.005407,NaN
3471,2023-02-15 09:35:00,2.785,2.791,2.780,2.781,22445477,21992500.0,2.760,NaN,NaN,0.020597,0.020597,0.020597,-0.007551,NaN,NaN
3519,2023-02-16 09:35:00,2.764,2.768,2.759,2.760,23858300,22445477.0,2.759,NaN,NaN,0.062945,0.062945,0.062945,-0.000362,NaN,NaN


In [72]:
np.mean(first5min_pd["volume_change"])
np.var(first5min_pd["volume_change"])

0.6913471959422548

In [73]:
a1 = np.mean(first5min_pd["close_change1"])
a2 = np.mean(first5min_pd["close_change2"])
a3 = np.mean(first5min_pd["close_change3"])
a5 = np.mean(first5min_pd["close_change5"])
a10 = np.mean(first5min_pd["close_change10"])
[a1, a2, a3, a5, a10]

[0.001995487517669501,
 0.00440816435537164,
 0.006823826237551712,
 0.010809396715423742,
 0.019872475614661277]

In [74]:
first5min_choosefangliang_pd = first5min_pd[first5min_pd["volume_change"] > 0.1 ]
first5min_choosefangliang_pd = first5min_choosefangliang_pd[first5min_choosefangliang_pd["close"] < first5min_choosefangliang_pd["open"] ]

In [66]:
first5min_choosefangliang_pd

,day,open,high,low,close,volume,volume_yestday,close_1day,close_3day,close_10day,volume_changge,volume_chang,volume_change,close_change1,close_change3,close_change10
63,2022-10-31 09:35:00,2.344,2.348,2.326,2.332,112566800,67370900.0,2.356,2.440,2.606,0.670852,0.670852,0.670852,0.010292,0.046312,0.117496
447,2022-11-10 09:35:00,2.464,2.465,2.453,2.460,44144500,36629100.0,2.547,2.565,2.596,0.205176,0.205176,0.205176,0.035366,0.042683,0.055285
495,2022-11-11 09:35:00,2.550,2.565,2.546,2.547,241306496,44144500.0,2.606,2.617,2.557,4.466287,4.466287,4.466287,0.023165,0.027483,0.003926
735,2022-11-18 09:35:00,2.605,2.610,2.598,2.601,62180200,31233700.0,2.542,2.577,2.631,0.990805,0.990805,0.990805,-0.022684,-0.009227,0.011534
783,2022-11-21 09:35:00,2.568,2.570,2.526,2.542,149844800,62180200.0,2.558,2.596,2.645,1.409848,1.409848,1.409848,0.006294,0.021243,0.040519
1023,2022-11-28 09:35:00,2.537,2.537,2.518,2.520,184660100,41745000.0,2.587,2.669,2.692,3.423526,3.423526,3.423526,0.026587,0.059127,0.068254
1263,2022-12-05 09:35:00,2.648,2.659,2.638,2.645,68163248,41311538.0,2.665,2.667,2.701,0.649981,0.649981,0.649981,0.007561,0.008318,0.021172
1407,2022-12-08 09:35:00,2.668,2.676,2.666,2.667,38422000,24011984.0,2.679,2.679,2.629,0.600118,0.600118,0.600118,0.004499,0.004499,-0.014248
1935,2022-12-23 09:35:00,2.614,2.617,2.605,2.608,68891740,51047342.0,2.626,2.641,2.740,0.349566,0.349566,0.349566,0.006902,0.012653,0.050613
2031,2022-12-27 09:35:00,2.639,2.639,2.628,2.631,57293854,28591926.0,2.641,2.653,2.750,1.003847,1.003847,1.003847,0.003801,0.008362,0.045230


In [76]:
a1 = np.mean(first5min_choosefangliang_pd["close_change1"])
a2 = np.mean(first5min_choosefangliang_pd["close_change2"])
a3 = np.mean(first5min_choosefangliang_pd["close_change3"])
a5 = np.mean(first5min_choosefangliang_pd["close_change5"])
a10 = np.mean(first5min_choosefangliang_pd["close_change10"])

[a1, a2, a3, a5, a10]

[0.007123177072801604,
 0.010923539356393269,
 0.013550902601279452,
 0.014818983272768897,
 0.02850013554524024]

In [195]:
sh50_pd["cc"] = sh50_pd["close"].shift()

In [197]:
sh50_pd["cc"] = (sh50_pd["close_5day"] - sh50_pd["close"])/sh50_pd["close"]

In [199]:
first5min_pd

,day,open,high,low,close,volume,volume_change
15,2022-10-28 09:35:00,2.403,2.413,2.393,2.393,67370900,NaN
63,2022-10-31 09:35:00,2.344,2.348,2.326,2.332,112566800,0.670852
111,2022-11-01 09:35:00,2.342,2.362,2.342,2.356,116989143,0.039286
159,2022-11-02 09:35:00,2.422,2.425,2.415,2.423,70515372,-0.397249
207,2022-11-03 09:35:00,2.438,2.445,2.432,2.440,66175229,-0.061549
...,...,...,...,...,...,...,...
3375,2023-02-13 09:35:00,2.746,2.753,2.738,2.752,34180900,0.040411
3423,2023-02-14 09:35:00,2.776,2.784,2.768,2.774,21992500,-0.356585
3471,2023-02-15 09:35:00,2.785,2.791,2.780,2.781,22445477,0.020597
3519,2023-02-16 09:35:00,2.764,2.768,2.759,2.760,23858300,0.062945


In [201]:
first5min_pd["volume_change"] = first5min_pd["volume"].pct_change()

In [210]:
first5min_pd = pd.DataFrame(sh50_pd.iloc[0::48, :])
first5min_pd = first5min_pd.reset_index()
first5min_pd


,index,day,open,high,low,close,volume,close_yes,close_2hour,close_1day,close_2day,close_3day,close_5day,close_10day,cc
0,15,2022-10-28 09:35:00,2.247,2.259,2.241,2.245,25448000,2.265,2.220,2.187,2.210,2.273,2.324,2.355,0.035189
1,63,2022-10-31 09:35:00,2.166,2.189,2.164,2.187,40535100,2.187,2.210,2.210,2.273,2.297,2.395,2.342,0.095107
2,111,2022-11-01 09:35:00,2.208,2.218,2.208,2.210,66824300,2.202,2.225,2.273,2.297,2.324,2.380,2.294,0.076923
3,159,2022-11-02 09:35:00,2.261,2.275,2.256,2.273,44424126,2.275,2.315,2.297,2.324,2.395,2.351,2.356,0.034316
4,207,2022-11-03 09:35:00,2.283,2.303,2.283,2.297,43137843,2.305,2.294,2.324,2.395,2.380,2.318,2.312,0.009142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,3375,2023-02-13 09:35:00,2.463,2.483,2.463,2.483,54181000,2.467,2.483,2.495,2.493,2.465,NaN,NaN,NaN
71,3423,2023-02-14 09:35:00,2.502,2.508,2.494,2.495,20604400,2.495,2.482,2.493,2.465,2.430,NaN,NaN,NaN
72,3471,2023-02-15 09:35:00,2.488,2.498,2.486,2.493,18856300,2.490,2.475,2.465,2.430,NaN,NaN,NaN,NaN
73,3519,2023-02-16 09:35:00,2.473,2.474,2.462,2.465,19104200,2.468,2.486,2.430,NaN,NaN,NaN,NaN,NaN


In [ ]:
def day_data(data_df):
    first5min_pd = data_df.iloc[0::48, :]
    first5min_pd["volume_change"] = first5min_pd["volume"].pct_change()
    first5min_pd["close_change_yes"] = (first5min_pd["open"] - first5min_pd["close_yes"])/first5min_pd["close_yes"]
    first5min_pd["close_change_2h"] = (first5min_pd["close_2hour"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change1"] = (first5min_pd["close_1day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change2"] = (first5min_pd["close_2day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change3"] = (first5min_pd["close_3day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change5"] = (first5min_pd["close_5day"] - first5min_pd["close"])/first5min_pd["close"]
    first5min_pd["close_change10"] = (first5min_pd["close_10day"] - first5min_pd["close"])/first5min_pd["close"]
    
    return first5min_pd